<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de sa page web).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

> Veuillez répondre ici (en commentaire) à la question.

a. Voici l'URL du ZIP : https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz <br>Voici l'URL du fichier : https://www.d.umn.edu/~tpederse/Data/README.int.txt

b. C'est un fichier texte où chaque phrase est délimitée par une ligne contenant "$$", chaque sens est annoté comme ceci interest_{numéro du sens}

c. Oui, le pluriel est le singulier sont traités

d. Une occurrence par phrase est annotée avec un tag de sens (ex: _1 à _6). Les autres occurrences du mot "interest" dans la même phrase sont marquées par un astérisque (*) placé juste avant le mot (ex: *interest) pour indiquer qu'elles ne sont pas l'objet de l'annotation de sens pour cette instance spécifique du corpus. Cela permet de respecter la consigne "une occurrence étiquetée par phrase" tout en conservant le contexte phrastique complet.

**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

> Veuillez répondre ici (en commentaire) à la question.
```text
Sense 1 =  361 occurrences (15%) - readiness to give attention
Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
Sense 5 =  500 occurrences (21%) - a share in a company or business
Sense 6 = 1252 occurrences (53%) - money paid for the use of money
	  ----
	  2368 occurrences in the sense tagged corpus, where each
	occurrence is a single sentence that contains the word 'interest'.
```

**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

> Veuillez répondre ici (en commentaire) à la question.

Le dictionnaire est "The first edition of Longman's Dictionary of Contemporary English", on peut le consulter ici https://www.ldoceonline.com/dictionary/
```text
Sense 1 = "if you have an interest in something or someone, you want to know or learn more about them"
Sense 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
Sense 4 = "the things that bring advantages to someone or something"
Sense 5 = "if you have an interest in a particular company or industry, you own shares in it"
Sense 6 = "the extra money that you must pay back when you borrow money" & "money paid to you by a bank or financial institution when you keep money in an account there"
```

**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

> Veuillez répondre ici (en commentaire) à la question.

Il y a 7 synsets: 

```text
Sense 1 = "a sense of concern with and curiosity about someone or something"
Sense 2 = "the power of attracting or holding one's attention (because it is unusual or exciting etc.)"
Sense 3 = "a diversion that occupies one's time and thoughts (usually pleasantly)"
Sense 4 = "a reason for wanting something done"
Sense 5 = "a right or legal share of something; a financial involvement with something"
Sense 6 = "a fixed charge for borrowing money; usually a percentage of the amount borrowed"
```

On a ignoré le sens suivant : "(usually plural) a social group whose members control some field of activity and who have common aims"

**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [ ]:
import nltk

In [ ]:
from random import randrange

In [ ]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
senses1 = [
    ['readiness', 'give', 'attention'],
    ['quality', 'causing', 'attention', 'given'],
    ['activity', 'attention'],
    ['advantage', 'advancement', 'favor'],
    ['share', 'company', 'business'],
    ['money', 'paid', 'money']
]
print(senses1)

**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

try:
    word_tokenize("test")
except LookupError:
    nltk.download('punkt')

LDOCE = [
    "if you have an interest in something or someone, you want to know or learn more about them",
    "a quality or feature of something that attracts your attention or makes you want to know more about it",
    "an activity that you enjoy doing or a subject that you enjoy studying",
    "the things that bring advantages to someone or something",
    "if you have an interest in a particular company or industry, you own shares in it",
    "the extra money that you must pay back when you borrow money" + " " + "money paid to you by a bank or financial institution when you keep money in an account there"
]

WNET = [
    "a sense of concern with and curiosity about someone or something",
    "the power of attracting or holding one's attention (because it is unusual or exciting etc.)",
    "a diversion that occupies one's time and thoughts (usually pleasantly)",
    "a reason for wanting something done",
    "a right or legal share of something; a financial involvement with something",
    "a fixed charge for borrowing money; usually a percentage of the amount borrowed"
]

combined_definitions_text = []
for i in range(len(LDOCE)):
    combined_definitions_text.append(LDOCE[i] + " " + WNET[i])

senses2 = []

stop_words = set(stopwords.words('english'))
stop_words.add('etc')
punctuation_chars = set(string.punctuation)


for definition_text in combined_definitions_text:
    text_lower = definition_text.lower()

    tokens = word_tokenize(text_lower)

    keywords_for_this_sense = []
    for token in tokens:
        if token.isalpha() and token not in stop_words:
            keywords_for_this_sense.append(token)

    unique_keywords = sorted(list(set(keywords_for_this_sense)))
    senses2.append(unique_keywords)

print(senses2)

**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [ ]:
# Veuillez répondre ici à la question.
sentences = []

filename = "interest-original.txt"

sentences = []
raw_lines_count = 0
with open(filename, 'r', encoding='utf-8') as f:
    current_sentence_words = []
    for line in f:
        raw_lines_count += 1
        line_stripped = line.strip()

        if line_stripped == "$$":
            if current_sentence_words:
                sentences.append(current_sentence_words)
                current_sentence_words = []
        elif line_stripped.startswith("======================================"):
            continue
        elif line_stripped:
            words_in_line = line_stripped.split()
            current_sentence_words.extend(words_in_line)

    if current_sentence_words:
        sentences.append(current_sentence_words)
        
print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])

## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [ ]:
# Veuillez répondre ici à la question.

from nltk.corpus import stopwords
import random

def wsd_lesk(senses, sentence, window_size=5):
    stop_words = set(stopwords.words('english'))

    target_word_index = -1
    for i, token in enumerate(sentence):
        if token.startswith("interest_") or token.startswith("interests_"):
            target_word_index = i
            break

    if target_word_index == -1:
        return None

    start_index = max(0, target_word_index - window_size)
    end_index = min(len(sentence), target_word_index + window_size + 1)
    context_tokens_raw = sentence[start_index:target_word_index] + sentence[target_word_index+1:end_index]

    context_cleaned = set()
    for token in context_tokens_raw:
        token_lower = token.lower()
        if token_lower.isalpha() and token_lower not in stop_words:
            context_cleaned.add(token_lower)
    
    if not context_cleaned:
        return random.randint(1, len(senses))

    best_sense_indices = []
    max_overlap = -1

    for i, sense_kws in enumerate(senses):
        sense_kws_set = set(sense_kws)
        overlap = len(context_cleaned.intersection(sense_kws_set))

        if overlap > max_overlap:
            max_overlap = overlap
            best_sense_indices = [i + 1]
        elif overlap == max_overlap and max_overlap != -1:
            best_sense_indices.append(i + 1)
    
    if not best_sense_indices:
        return random.randint(1, len(senses))
    elif len(best_sense_indices) == 1:
        return best_sense_indices[0]
    else:
        return random.choice(best_sense_indices) # Tirer la réponse au sort parmi les meilleurs


print(wsd_lesk(senses1, sentences[11]))
print(wsd_lesk(senses2, sentences[11]))

print(senses2[4])
print(sentences[11])

**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [ ]:
# Veuillez répondre ici à la question.
import random
from nltk.corpus import stopwords # Nécessaire si wsd_lesk l'utilise en interne

def evaluate_wsd(wsd_function, senses_keywords, sentences_data, window_size=5):
    correct_predictions = 0
    total_predictions = 0

    for sentence_tokens in sentences_data:
        true_sense = None
        for token in sentence_tokens:
            if token.startswith("interest_") or token.startswith("interests_"):
                try:
                    sense_part = token.split('_')[-1]
                    numeric_sense_str = ""
                    for char in sense_part:
                        if char.isdigit():
                            numeric_sense_str += char
                        else:
                            break
                    if numeric_sense_str:
                        true_sense = int(numeric_sense_str)
                    break
                except (IndexError, ValueError):
                    continue

        if true_sense is None:
            continue

        total_predictions += 1
       
        predicted_sense = wsd_function(senses_keywords, sentence_tokens, window_size)
        
        if predicted_sense is not None and predicted_sense == true_sense:
            correct_predictions += 1

    if total_predictions == 0:
        return 0.0

    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

score = evaluate_wsd(wsd_lesk, senses2, sentences)
print(f"Score de la méthode 'wsd_lesk': {score:.2f}%")

**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [ ]:
# On va itérer sur 20 évaluations pour chaque taille de fenetre pour les deux listes (senses1 et senses2) pour avoir la moyenne des meilleurs scores
window_sizes_to_test = [5, 7, 10, 12, 15, 17, 20]
num_runs_for_averaging = 20

results = {}

for sense_list_name, sense_list_data in [("senses1", senses1), ("senses2", senses2)]:
    print(f"Évaluation pour {sense_list_name}:")
    best_avg_score_for_senselist = -1
    best_window_for_senselist = -1
    all_scores_for_windows = {}
    
    for ws in window_sizes_to_test:
        current_ws_scores = []
        for run in range(num_runs_for_averaging):
            score = evaluate_wsd(wsd_lesk, sense_list_data, sentences, window_size=ws)
            current_ws_scores.append(score)
        
        avg_score = sum(current_ws_scores) / len(current_ws_scores) if current_ws_scores else 0
        all_scores_for_windows[ws] = avg_score
        print(f"  Window size {ws}: Moyenne score sur {num_runs_for_averaging} exécutions = {avg_score:.2f}%")
        
        if avg_score > best_avg_score_for_senselist:
            best_avg_score_for_senselist = avg_score
            best_window_for_senselist = ws
            
    results[sense_list_name] = {
        'best_avg_score': best_avg_score_for_senselist,
        'best_window': best_window_for_senselist,
        'all_scores': all_scores_for_windows
    }

best_overall_score = -1
best_overall_setup = ""

for sense_name, data in results.items():
    print(f"Pour {sense_name}: Meilleur score moyen = {data['best_avg_score']:.2f}% (fenêtre = {data['best_window']})")
    if data['best_avg_score'] > best_overall_score:
        best_overall_score = data['best_avg_score']
        best_overall_setup = f"avec {sense_name} et une fenêtre de {data['best_window']}"

print(f"\nLe meilleur score global (moyenné) de la méthode de Lesk simplifiée est : {best_overall_score:.2f}% {best_overall_setup}.")

if results.get("senses2", {}).get('best_avg_score', -1) > results.get("senses1", {}).get('best_avg_score', -1):
    print("La liste de sens 'senses2' conduit à de meilleurs scores moyens.")
elif results.get("senses1", {}).get('best_avg_score', -1) > results.get("senses2", {}).get('best_avg_score', -1):
    print("La liste de sens 'senses1' conduit à de meilleurs scores moyens.")
else:
    if results.get("senses1", {}).get('best_avg_score', -1) > -1 : # Vérifier qu'on a bien eu des scores
        print("Les deux listes de sens conduisent à des scores moyens similaires (en considérant leur meilleure fenêtre respective).")
    else:
        print("Impossible de déterminer quelle liste de sens est meilleure en raison de données/scores manquants.")

L'algorithme n'est pas déterministe, donc c'est difficile de savoir quel est la meilleure taille de la fenêtre. `senses2` conduit à des meilleurs scores et notre meilleur score obtenu est autour de 28% pour une taille de fenêtre de 17.

Détail des résultats:

```
Évaluation pour senses1:
  Window size 5: Moyenne score sur 20 exécutions = 19.71%
  Window size 7: Moyenne score sur 20 exécutions = 19.74%
  Window size 10: Moyenne score sur 20 exécutions = 20.39%
  Window size 12: Moyenne score sur 20 exécutions = 20.62%
  Window size 15: Moyenne score sur 20 exécutions = 20.70%
  Window size 17: Moyenne score sur 20 exécutions = 20.81%
  Window size 20: Moyenne score sur 20 exécutions = 21.37%
Évaluation pour senses2:
  Window size 5: Moyenne score sur 20 exécutions = 25.61%
  Window size 7: Moyenne score sur 20 exécutions = 26.21%
  Window size 10: Moyenne score sur 20 exécutions = 27.07%
  Window size 12: Moyenne score sur 20 exécutions = 27.40%
  Window size 15: Moyenne score sur 20 exécutions = 27.59%
  Window size 17: Moyenne score sur 20 exécutions = 28.08%
  Window size 20: Moyenne score sur 20 exécutions = 28.05%

Pour senses1: Meilleur score moyen = 21.37% (fenêtre = 20)
Pour senses2: Meilleur score moyen = 28.08% (fenêtre = 17)
```

## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.  

In [ ]:
import gensim
from gensim.models import KeyedVectors
import gensim.downloader as api
# path_to_model = "../../../gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz"
# wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)
wv_model = api.load("word2vec-google-news-300")

In [ ]:
# Veuillez répondre ici à la question.
def wsd_word2vec(senses, sentence, window_size=5):
    stop_words = set(stopwords.words('english'))

    # Identify target word and its position
    target_word_index = -1
    target_word = None
    for i, token in enumerate(sentence):
        if token.startswith("interest_") or token.startswith("interests_"):
            target_word_index = i
            target_word = token.split("_")[0]  # Get the actual word: "interest"
            break

    if target_word_index == -1 or target_word not in wv_model:
        return random.randint(1, len(senses)) if senses else None

    # Extract context tokens around the target word
    start_index = max(0, target_word_index - window_size)
    end_index = min(len(sentence), target_word_index + window_size + 1)
    context_tokens_raw = sentence[start_index:target_word_index] + sentence[target_word_index+1:end_index]

    # Clean context tokens
    context_cleaned = [
        token.lower() for token in context_tokens_raw
        if token.isalpha() and token.lower() not in stop_words and token.lower() in wv_model
    ]

    if not context_cleaned:
        return random.randint(1, len(senses)) if senses else None
    
    best_sense_indices = []
    max_similarity = -float('inf') # Initialize with low value

    for i, sense_kws in enumerate(senses):
            sense_kws_in_vocab = [kw for kw in sense_kws if kw in wv_model]

            if not sense_kws_in_vocab:
                similarity = 0.0
            else:
                try:
                    similarity = wv_model.n_similarity(context_cleaned, sense_kws_in_vocab)
                except Exception as e:
                    similarity = 0.0 

            if similarity > max_similarity:
                max_similarity = similarity
                best_sense_indices = [i + 1]
            elif similarity == max_similarity:
                if max_similarity > -float('inf'):
                    best_sense_indices.append(i + 1)

    if not best_sense_indices:
        # No sense has been compared or every senses have no or a low similarity
        return random.randint(1, len(senses)) if senses else None
    elif len(best_sense_indices) == 1:
        return best_sense_indices[0]
    else:
        # Equality, random choose bewtween the best
        return random.choice(best_sense_indices)

**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [ ]:
# Veuillez répondre ici à la question.

window_sizes_to_test_w2v = [5, 7, 10, 12, 15, 17, 20]
num_runs_for_averaging_w2v = 20 
results_w2v = {}

for sense_list_name, sense_list_data in [("senses1", senses1), ("senses2", senses2)]:
    print(f"Évaluation de wsd_word2vec pour {sense_list_name}:")
    best_avg_score_for_senselist = -1.0
    best_window_for_senselist = -1
    all_scores_for_windows = {}

    for ws in window_sizes_to_test_w2v:
        current_ws_scores = []
        for run in range(num_runs_for_averaging_w2v):
            score = evaluate_wsd(wsd_word2vec, sense_list_data, sentences, window_size=ws)
            current_ws_scores.append(score)
        
        avg_score = sum(current_ws_scores) / len(current_ws_scores) if current_ws_scores else 0.0
        all_scores_for_windows[ws] = avg_score
        print(f"  Window size {ws}: Moyenne score sur {num_runs_for_averaging_w2v} exécutions = {avg_score:.2f}%")
        
        if avg_score > best_avg_score_for_senselist:
            best_avg_score_for_senselist = avg_score
            best_window_for_senselist = ws
            
    results_w2v[sense_list_name] = {
        'best_avg_score': best_avg_score_for_senselist,
        'best_window': best_window_for_senselist,
        'all_scores': all_scores_for_windows
    }

# Affichage et comparaison des meilleurs scores pour wsd_word2vec
best_overall_score_w2v = -1.0
best_overall_setup_w2v = ""
s1_data_w2v = results_w2v.get("senses1", {'best_avg_score': -1.0, 'best_window': 'N/A'})
s2_data_w2v = results_w2v.get("senses2", {'best_avg_score': -1.0, 'best_window': 'N/A'})
if s1_data_w2v['best_avg_score'] > -1.0 :
    print(f"Pour senses1 avec wsd_word2vec: Meilleur score moyen = {s1_data_w2v['best_avg_score']:.2f}% (fenêtre = {s1_data_w2v['best_window']})")
    if s1_data_w2v['best_avg_score'] > best_overall_score_w2v:
        best_overall_score_w2v = s1_data_w2v['best_avg_score']
        best_overall_setup_w2v = f"avec senses1 et une fenêtre de {s1_data_w2v['best_window']}"
if s2_data_w2v['best_avg_score'] > -1.0 :
    print(f"Pour senses2 avec wsd_word2vec: Meilleur score moyen = {s2_data_w2v['best_avg_score']:.2f}% (fenêtre = {s2_data_w2v['best_window']})")
    if s2_data_w2v['best_avg_score'] > best_overall_score_w2v:
        best_overall_score_w2v = s2_data_w2v['best_avg_score']
        best_overall_setup_w2v = f"avec senses2 et une fenêtre de {s2_data_w2v['best_window']}"
if best_overall_score_w2v > -1.0:
    print(f"\nLe meilleur score global (moyenné) pour wsd_word2vec est : {best_overall_score_w2v:.2f}% {best_overall_setup_w2v}.")
    if s2_data_w2v['best_avg_score'] > s1_data_w2v['best_avg_score']:
        print("La liste de sens 'senses2' conduit à de meilleurs scores moyens avec wsd_word2vec.")
    elif s1_data_w2v['best_avg_score'] > s2_data_w2v['best_avg_score']:
        print("La liste de sens 'senses1' conduit à de meilleurs scores moyens avec wsd_word2vec.")
    else:
        print("Les deux listes de sens conduisent à des scores moyens similaires avec wsd_word2vec.")

La méthode wsd_word2vec surpasse la méthode de Lesk simplifiée. La précision passe d'environ 28% pour Lesk à environ 64% pour Word2Vec.

L'utilisation des embeddings de mots pour capturer la similarité entre le contexte et les définitions des sens est beaucoup plus efficace que le simple comptage de mots en commun. Dans les deux cas, la liste de mots-clés senses2 a conduit à de meilleurs scores que senses1. La quautité et la qualité des mots-clés permettent une meilleure performance.

Détail des résultats:

```
Évaluation de wsd_word2vec pour senses1:
  Window size 5: Moyenne score sur 20 exécutions = 39.01%
  Window size 7: Moyenne score sur 20 exécutions = 39.90%
  Window size 10: Moyenne score sur 20 exécutions = 40.23%
  Window size 12: Moyenne score sur 20 exécutions = 41.45%
  Window size 15: Moyenne score sur 20 exécutions = 40.15%
  Window size 17: Moyenne score sur 20 exécutions = 40.88%
  Window size 20: Moyenne score sur 20 exécutions = 41.29%

Évaluation de wsd_word2vec pour senses2:
  Window size 5: Moyenne score sur 20 exécutions = 63.60%
  Window size 7: Moyenne score sur 20 exécutions = 63.11%
  Window size 10: Moyenne score sur 20 exécutions = 62.13%
  Window size 12: Moyenne score sur 20 exécutions = 64.01%
  Window size 15: Moyenne score sur 20 exécutions = 62.95%
  Window size 17: Moyenne score sur 20 exécutions = 62.79%
  Window size 20: Moyenne score sur 20 exécutions = 63.52%
Pour senses1 avec wsd_word2vec: Meilleur score moyen = 41.45% (fenêtre = 12)
Pour senses2 avec wsd_word2vec: Meilleur score moyen = 64.01% (fenêtre = 12)

Le meilleur score global (moyenné) pour wsd_word2vec est : 64.01% avec senses2 et une fenêtre de 12.
La liste de sens 'senses2' conduit à de meilleurs scores moyens avec wsd_word2vec.
```

## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le premier but sera de transformer chaque phrase en un ensemble d'attributs pour formater les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

On vous propose de nommer les attributs `word-k`, ..., `word-2`, `word-1`, `word+1`, `word+2`, ..., `word+k` (fenêtre de taille `2*k` autour de *interest*).  Leurs valeurs sont les mots observés aux emplacements respectifs, ou `NONE` si la position dépasse l'étendue de la phrase.  Vous ajouterez un attribut nommé `word0` qui est l'occurrence du mot *interest* au singulier ou au pluriel.  

Pour chaque occurrence de *interest*, vous devrez donc créer la représentation suivante (où `6` est le numéro du sens, essentiel pour l'entraînement, mais à cacher lors de l'évaluation) :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```

**4a.** En partant de la liste des phrases appelée `sentences` préparée plus haut, veuillez générer la liste avec toutes les représentation, appelée `items_with_features`.  Vous pouvez vous aider du livre NLTK.

In [ ]:
# Veuillez répondre ici à la question.

items_with_features = []
window_half_size = 2 # Pour avoir word-2, word-1, word+1, word+2 (k=2)

for sentence_idx, sentence_tokens in enumerate(sentences):
    target_indices = []
    # Trouver tous les index des occurrences de 'interest_X' ou 'interests_X'
    for token_idx, token in enumerate(sentence_tokens):
        if token.startswith("interest_") or token.startswith("interests_"):
            target_indices.append(token_idx)

    for target_idx in target_indices:
        numeric_sense = None
        target_token_full = sentence_tokens[target_idx] # ex: interest_6
        
        try:
            sense_part = target_token_full.split('_')[-1]
            # Extraire uniquement la partie numérique du sens
            numeric_sense_str = "".join(filter(str.isdigit, sense_part))
            if numeric_sense_str: # S'assurer qu'on a bien extrait des chiffres
                numeric_sense = int(numeric_sense_str)
            else:
                continue
        except (IndexError, ValueError):
            continue

        features = {}
        
        # On prend le mot avant le _ (interest ou interests)
        features['word0'] = target_token_full.split('_')[0]

        # Attributs word-k à word+k
        for k_offset in range(1, window_half_size + 1):
            # Contexte avant (word-k)
            prev_idx = target_idx - k_offset
            if prev_idx >= 0:
                features[f'word-{k_offset}'] = sentence_tokens[prev_idx].lower()
            else:
                features[f'word-{k_offset}'] = 'NONE'

            # Contexte après (word+k)
            next_idx = target_idx + k_offset
            if next_idx < len(sentence_tokens):
                features[f'word+{k_offset}'] = sentence_tokens[next_idx].lower()
            else:
                features[f'word+{k_offset}'] = 'NONE'
        
        items_with_features.append((features, numeric_sense))
        
print(f"Nombre total d'items avec features générés : {len(items_with_features)}")
if len(items_with_features) > 153:
    print("\nExemples d'items_with_features :")
    for i in range(151, min(154, len(items_with_features))):
        print(items_with_features[i])
else:
    print("\nMoins de 154 items générés, affichage des premiers:")
    for i in range(min(3, len(items_with_features))):
        print(items_with_features[i])

**4b.** Veuillez séparer les données aléatoirement en 80% pour l'entraînement et 20%  pour l'évaluation.  Veuillez faire une division stratifiée : les deux sous-ensembles doivent contenir les mêmes proportions de sens que l'ensemble de départ.  Ils seront appelés `iwf_train` et `iwf_test`.

In [ ]:
from collections import defaultdict
from random import shuffle

In [ ]:
iwf_train = []
iwf_test  = []

# Veuillez répondre ici à la question.

def stratified_split(items_with_features_list, train_ratio=0.8):

    train_set = []
    test_set = []
    
    items_by_label = defaultdict(list)
    for item in items_with_features_list:
        # S'assurer que l'item est bien un tuple (ou liste) de 2 éléments
        if isinstance(item, (list, tuple)) and len(item) == 2:
            items_by_label[item[1]].append(item) # item[1] est le label
        else:
            # Gérer le cas où un item n'a pas le bon format, si nécessaire
            pass

    for label, items in items_by_label.items():
        shuffle(items) # Mélanger au sein de la classe
        
        # Calcul du point de partage
        split_idx = int(len(items) * train_ratio)
        
        train_set.extend(items[:split_idx])
        test_set.extend(items[split_idx:])
    
    shuffle(train_set) # Mélanger l'ensemble d'entraînement final
    shuffle(test_set)  # Mélanger l'ensemble de test final
    return train_set, test_set

iwf_train, iwf_test = stratified_split(items_with_features)
print(f"\nTaille de l'ensemble d'entraînement : {len(iwf_train)}")
print(f"Taille de l'ensemble de test : {len(iwf_test)}")

**4c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_train` et la tester sur `iwf_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score que vous obtenez (avec la fonction `accuracy` de NLTK) sur l'ensemble de test ?  Comment se compare-t-il avec les précédents ?

In [ ]:
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy

# Génère la liste des (featureset, label) pour la classification.
def generate_features_for_classification(sentences_data, window_k):
    items = []
    for sentence_tokens in sentences_data:
        target_indices = [i for i, token in enumerate(sentence_tokens)
                          if token.startswith("interest_") or token.startswith("interests_")]

        for target_idx in target_indices:
            numeric_sense = None
            target_token_full = sentence_tokens[target_idx]
            sense_part = target_token_full.split('_')[-1]
            numeric_sense_str = "".join(filter(str.isdigit, sense_part))
            if numeric_sense_str: numeric_sense = int(numeric_sense_str)
            else: continue

            features = {}
            features['word0'] = target_token_full.split('_')[0].lower()

            for k_offset in range(1, window_k + 1):
                features[f'word-{k_offset}'] = sentence_tokens[target_idx - k_offset].lower() if (target_idx - k_offset) >= 0 else 'NONE'
                features[f'word+{k_offset}'] = sentence_tokens[target_idx + k_offset].lower() if (target_idx + k_offset) < len(sentence_tokens) else 'NONE'
            
            items.append((features, numeric_sense))
    return items

# Définir les tailles de demi-fenêtre (k) à tester
window_k_values_to_test = [1, 2, 3, 4, 5, 7, 10] 
best_nb_accuracy = -1.0
best_nb_window_k = -1
for k_val in window_k_values_to_test:
    current_items_with_features = generate_features_for_classification(sentences, window_k=k_val)
    if not current_items_with_features:
        continue

    # Division stratifiée
    iwf_train, iwf_test = stratified_split(current_items_with_features, train_ratio=0.8)
    if not iwf_train or not iwf_test:
        continue
    
    # Entraînement du classifieur NaiveBayes
    classifier_nb = NaiveBayesClassifier.train(iwf_train)

    # Test du classifieur
    current_accuracy = accuracy(classifier_nb, iwf_test) * 100 # en pourcentage
    print(f"  Accuracy pour k={k_val}: {current_accuracy:.2f}%")
    if current_accuracy > best_nb_accuracy:
        best_nb_accuracy = current_accuracy
        best_nb_window_k = k_val

print(f"\n--- Résultat pour NaiveBayesClassifier ---")
if best_nb_window_k != -1:
    print(f"Meilleur score d'accuracy obtenu : {best_nb_accuracy:.2f}%")
    print(f"Avec une demi-taille de fenêtre (k) = {best_nb_window_k} (donc {2*best_nb_window_k} mots de contexte + word0).")

print(f"\n--- Comparaison des scores ---")
best_lesk_score = 28.08
best_w2v_score = 64.01
print(f"Meilleur score Lesk simplifié : {best_lesk_score:.2f}%")
print(f"Meilleur score Word2Vec       : {best_w2v_score:.2f}%")
if best_nb_window_k != -1:
    print(f"Meilleur score NaiveBayes     : {best_nb_accuracy:.2f}%")
    if best_nb_accuracy > best_w2v_score and best_nb_accuracy > best_lesk_score:
        print("\nNaiveBayes a obtenu le meilleur score global.")
    elif best_w2v_score > best_nb_accuracy and best_w2v_score > best_lesk_score:
        print("\nWord2Vec a obtenu le meilleur score global.")
    elif best_lesk_score > best_nb_accuracy and best_lesk_score > best_w2v_score:
        print("\nLesk simplifié a obtenu le meilleur score global.")
    else:
        # Gérer les autres cas d'égalité ou de classement
        scores_dict = {"Lesk": best_lesk_score, "Word2Vec": best_w2v_score, "NaiveBayes": best_nb_accuracy}
        sorted_scores = sorted(scores_dict.items(), key=lambda item: item[1], reverse=True)
        print("\nClassement des méthodes :")
        for method, score_val in sorted_scores:
            print(f"  - {method}: {score_val:.2f}%")
else:
    print("Impossible de comparer avec NaiveBayes car aucun score valide n'a été obtenu.")

Meilleur score avec `accuracy` : 85.08% avec une demi-taille de fenêtre (k) = 3 (donc 6 mots de contexte + word0).

Comparaison des scores:
* Meilleur score Lesk simplifié : 28.08%
* Meilleur score Word2Vec       : 64.01%
* Meilleur score NaiveBayes     : 85.08%

NaiveBayes a obtenu le meilleur score global.

**4d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

In [ ]:
# Ré-entraînement du classifieur final avec la meilleure demi-fenêtre
window = 3
items_final_features = generate_features_for_classification(sentences, window)
iwf_train_final, iwf_test_final = stratified_split(items_final_features, train_ratio=0.8) 
classifier_final_nb = NaiveBayesClassifier.train(iwf_train_final)
print(f"  Accuracy pour k={window}: {accuracy(classifier_final_nb, iwf_test_final) * 100 :.2f}%")

num_informative_features_to_show = 10
print(f"\nLes {num_informative_features_to_show} attributs les plus informatifs (k={window}):")
classifier_final_nb.show_most_informative_features(num_informative_features_to_show)

Résultats:

```
Accuracy pour k=3: 84.68%

Les 10 attributs les plus informatifs (k=3):
Most Informative Features
                   word0 = 'interests'         3 : 1      =     68.7 : 1.0
                  word+1 = 'in'                1 : 6      =     54.2 : 1.0
                  word+1 = 'of'                4 : 6      =     37.7 : 1.0
                  word-1 = 'other'             3 : 6      =     19.9 : 1.0
                  word+2 = ','                 6 : 5      =     16.2 : 1.0
                  word-2 = 'have'              1 : 6      =     14.4 : 1.0
                  word-2 = 'company'           5 : 6      =     10.4 : 1.0
                  word-2 = 'NONE'              6 : 4      =     10.2 : 1.0
                  word+2 = 'the'               1 : 2      =      9.9 : 1.0
                  word-1 = 'in'                6 : 5      =      9.7 : 1.0
```

* word0 = 'interests' 3 : 1 = 68.7 :  Si le mot cible est "interests" (pluriel), il est plus probable que ce soit le sens 3 (activité/hobby) que le sens 1 (curiosité/attention). Le pluriel "interests" est associé aux activités (sens 5), tandis que le sens 1 est souvent au singulier.
* word+1 = 'in' 1 : 6 = 54.2 : Si le mot immédiatement après "interest(s)" est "in", il est très probable que ce soit le sens 1 plutôt que le sens 6 (financier). C'est cohérent avec la structure "interest in [quelque chose]".
* word+1 = 'of' 4 : 6 = 37.7 : Si le mot suivant est "of", il est bien plus probable que ce soit le sens 4 (avantage, "the interest of...") que le sens 6. Capture la structure "interest of [entité/groupe]" typique du sens 4.
* word-1 = 'other' 3 : 6 = 19.9 : Si le mot précédant "interest(s)" est "other", il est très probable que ce soit le sens 3 (activités/hobbies) plutôt que le sens 6.
* word+2 = ',' 6 : 5 = 16.2 : Si le mot en position +2 est une virgule, il est plus probable que ce soit le sens 6 (financier) que le sens 5 (part dans une entreprise). Suggère des structures de phrases où une information financière sur "interest" est suivie d'une clause (par exemple, "interest rates, which have fallen,...", "principal and interest, amounted to...").
* word-2 = 'have' 1 : 6 = 14.4 : Si deux mots avant "interest(s)" on a "have" (ex: "they have an interest in..."), il est plus probable que ce soit le sens 1 que le sens 6. La construction "to have an interest in" est typique du sens 1.
* word-2 = 'company' 5 : 6 = 10.4 : Si le mot deux positions avant "interest(s)" est "company" (ex: "the company's interest in..."), il est bien plus probable que ce soit le sens 5 (part dans une entreprise) que le sens 6.
* word-2 = 'NONE' 6 : 4 = 10.2 : Si "interest(s)" apparaît comme le deuxième mot de la phrase, il est plus probable que ce soit le sens 6 que le sens 4. NONE à word-2 signifie que interest(s) est précédé d'au plus un mot. Cela pourrait correspondre à des phrases commençant par "Interest rates..." ou "The interest was...".
* word+2 = 'the' 1 : 2 = 9.9 : Si deux mots après "interest(s)" on a "the" (ex: "interest in the outcome"), il est plus probable que ce soit le sens 1 que le sens 2. Le sens 2 étant de toute façon très rare et probablement mal prédit. "interest in the..." est une structure courante pour le sens 1.
* word-1 = 'in' 6 : 5 = 9.7 : Si le mot précédant "interest(s)" est "in" (ex: "decline in interest rates"), il est plus probable que ce soit le sens 6 (financier) que le sens 5 (part dans une entreprise).

Les features les plus informatives restent largement basées sur des mots fonctionnels dans des positions ou la forme du mot "interest" lui-même. Le fait que l'accuracy soit toujours élevée (84.68%) montre que ces features positionnelles, même simples, sont très efficaces pour cette tâche avec Naive Bayes.

**4e.** On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (voir le [livre NLTK](https://www.nltk.org/book/ch06.html)), et comptabiliser les prédictions correctes pour chaque sens.  Vous pouvez vous inspirer de `evaluate_wsd`, et écrire une fonction `evaluate_wsd_supervised(classifier, items_with_features)`, que vous appliquerez aux donnés `iwf_test`.  Veuillez afficher ces scores.

In [ ]:
def evaluate_wsd_supervised(classifier, items_with_features_test_set):
    true_positives = defaultdict(int)
    false_positives = defaultdict(int)
    false_negatives = defaultdict(int)
    
    reference_labels = []
    predicted_labels = []

    for featureset, true_label in items_with_features_test_set:
        predicted_label = classifier.classify(featureset)
        reference_labels.append(true_label)
        predicted_labels.append(predicted_label)

    all_labels = sorted(list(set(reference_labels) | set(predicted_labels)))
    if not all_labels:
        print("Aucun label trouvé dans les données de test ou les prédictions.")
        return

    for true_label, predicted_label in zip(reference_labels, predicted_labels):
        if true_label == predicted_label:
            true_positives[true_label] += 1
        else:
            false_positives[predicted_label] += 1
            false_negatives[true_label] += 1

    print(f"{'Sens'} | {'Précision'} | {'Rappel'} | {'Score F1'}")
    print("-" * 37)

    weighted_avg_f1_numerator = 0
    total_support = 0
    f1_scores_for_macro_avg = [] # Liste pour stocker les F1 scores de chaque classe

    for L in all_labels: # Renommé 'label' en 'L' pour éviter conflit avec 'label' de la boucle externe
        tp = true_positives[L]
        fp = false_positives[L]
        fn = false_negatives[L]
        
        support = tp + fn 

        precision_score = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall_score = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            
        if (precision_score + recall_score) == 0:
            f1_score = 0.0
        else:
            f1_score = 2 * (precision_score * recall_score) / (precision_score + recall_score)
        
        f1_scores_for_macro_avg.append(f1_score) # Ajouter le F1 score pour la moyenne macro
            
        print(f"{L:<4} | {precision_score:<9.2f} | {recall_score:<6.2f} | {f1_score:<7.2f}")

        weighted_avg_f1_numerator += f1_score * support
        total_support += support
    
    if total_support > 0:
        # Calcul du Macro Average F1 (moyenne simple des F1 scores)
        macro_avg_f1 = sum(f1_scores_for_macro_avg) / len(f1_scores_for_macro_avg) if f1_scores_for_macro_avg else 0.0
        
        # Calcul du Weighted Average F1
        weighted_avg_f1 = weighted_avg_f1_numerator / total_support
        
        print("-" * 37)
        print(f"Macro Avg F1: {macro_avg_f1:.2f}")
        print(f"Weighted Avg F1: {weighted_avg_f1:.2f}")

evaluate_wsd_supervised(classifier_final_nb, iwf_test_final)

Scores obtenus:
```
Sens | Précision | Rappel | Score F1
-------------------------------------
1    | 0.67      | 0.82   | 0.74   
2    | 0.00      | 0.00   | 0.00   
3    | 0.60      | 0.38   | 0.46   
4    | 0.62      | 0.78   | 0.69   
5    | 0.76      | 0.69   | 0.72   
6    | 0.99      | 0.94   | 0.96   
-------------------------------------
Macro Avg F1: 0.60
Weighted Avg F1: 0.85
```

Note concernant le sens 2: Ce sens est soit absent de l'ensemble de test, soit le classifieur ne l'a jamais correctement prédit (et n'a probablement jamais essayé de le prédire en raison de sa rareté dans l'ensemble d'entraînement). C'est un point faible évident, principalement dû au manque de données pour ce sens.

## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des trois expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

In [ ]:
#  Comparaison des scores:
# Meilleur score Lesk simplifié : 28.08%
# Meilleur score Word2Vec       : 64.01%
# Meilleur score NaiveBayes     : 85.08% --> Meilleur score obtenu

## Fin du laboratoire

Merci de nettoyer votre feuille, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.